In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import date
#_ = !pip install geopy pycountry-convert tqdm -q
import pycountry_convert as pc
import statsmodels.api as sm

In [ ]:
df=pd.read_csv('time_series_covid19_confirmed_global.csv')
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


Data seem at state granularity and time dimension and spans over columns, so we would expect each country to be assigned to a single row. As I wish to focus on European countries I will be exploiting Latitude and Longitude measures to filter only relevant observations (in Europe)

In [ ]:
#FUNCTION TO FIND CONTINENT BASED ON COUNTRY NAME WITH GEOPY
def executeFunction(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f'Exception caught for{args}') #Returns None if cannot map
    return wrapper

@executeFunction
def get_continent(country_name):
  c=pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(country_name))
  return pc.convert_continent_code_to_continent_name(c)


In [ ]:
df.insert(4,'Continent',df['Country/Region'].apply(lambda x: get_continent(x)))

Exception caught for('Antarctica',)
Exception caught for('Burma',)
Exception caught for('Congo (Brazzaville)',)
Exception caught for('Congo (Kinshasa)',)
Exception caught for("Cote d'Ivoire",)
Exception caught for('Diamond Princess',)
Exception caught for('Holy See',)
Exception caught for('Korea, North',)
Exception caught for('Korea, South',)
Exception caught for('Kosovo',)
Exception caught for('MS Zaandam',)
Exception caught for('Summer Olympics 2020',)
Exception caught for('Taiwan*',)
Exception caught for('Timor-Leste',)
Exception caught for('US',)
Exception caught for('West Bank and Gaza',)
Exception caught for('Winter Olympics 2022',)


None of exceptions concerns Europe but Kosovo & Holy See, since the latter is the 7th most densely populated state on earth but only counts less than 30 cases as of today I would drop it as an anomaly. In case I would want to merge the data with population density information this would represent as issue.
In the case of Kosovo I simply assign 'Europe' label under continent manually

In [ ]:
df.loc[df['Country/Region']=='Kosovo','Continent']='Europe'
df=df[df['Continent']=='Europe'].reset_index(drop=True)
df.head()

,Province/State,Country/Region,Lat,Long,Continent,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Albania,41.1533,20.168300,Europe,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
1,NaN,Andorra,42.5063,1.521800,Europe,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
2,NaN,Austria,47.5162,14.550100,Europe,0,0,0,0,0,...,5911294,5919616,5926148,5931247,5936666,5940935,5943417,5949418,5955860,5961143
3,NaN,Belarus,53.7098,27.953400,Europe,0,0,0,0,0,...,994037,994037,994037,994037,994037,994037,994037,994037,994037,994037
4,NaN,Belgium,50.8333,4.469936,Europe,0,0,0,0,0,...,4717655,4717655,4727795,4727795,4727795,4727795,4727795,4727795,4727795,4739365


We should check whether there's duplicates in the crossectional and time dimension. If there's no duplicate we can set it as an index,
otherwise I solve the data issue by aggregation considering the nature of the presence of duplicates countries. In the case

In [ ]:
continental_extremes={'N':(81.843056, 59.239444),
                      'S':(34.833333, 24.083333),
                      'W':(39.495222, -31.275),
                      'E':(75.5, 60)}

def within_area(coordinates_array):
  if coordinates_array[0]<continental_extremes['N'][0] and coordinates_array[0]>continental_extremes['S'][0]:
    if coordinates_array[1]>continental_extremes['W'][1] and coordinates_array[0]<continental_extremes['E'][1]:
      return True
    else:
      return False
  else:
    return False

P=df[df.duplicated(subset=['Country/Region'])]
P.insert(4,'WithinArea',
 [within_area(df[df.duplicated(subset=['Country/Region'])][['Lat','Long']].values[i])
  for i in range(0,df[df.duplicated(subset=['Country/Region'])].shape[0])])
P=P[P['WithinArea']==True].drop(columns=['Lat','Long','WithinArea']).groupby(['Country/Region']).sum()

df=df[df['Country/Region'].isin([*P.index])==False].drop(columns=['Province/State',
                                                               'Lat','Long','Continent']).set_index('Country/Region')

df=pd.concat([df,P],axis=0).T
df.index=pd.DatetimeIndex(df.index)
df=df.sort_index()

del P

<ipython-input-304-833667e3e7a5>:19: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



I follow a geopolitical rule. If the Province/State is part of Europe as a continent it will be taken into account and its cases will be summed up to the country's count of confirmed cases. I perform this check by considering extremal latitude & longitude with respect to the 4 cardinal points of the European continent. If the coordinates of that Province/State falls within the rectangular area spanned, then data is a aggregated.

We now check if dates are in order. Usually column names are coded as string, so we

In [ ]:
px.line(df)

In [ ]:
d1_df=df.diff().iloc[1:,:]

In [ ]:
d1_df

Country/Region,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Czechia,Estonia,...,Slovakia,Slovenia,Spain,Sweden,Switzerland,Ukraine,Denmark,France,Netherlands,United Kingdom
2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-05,0.0,0.0,4269.0,0.0,0.0,0.0,25.0,0.0,140.0,0.0,...,31.0,102.0,0.0,0.0,0.0,141.0,0.0,0.0,0.0,0.0
2023-03-06,0.0,0.0,2482.0,0.0,0.0,0.0,181.0,334.0,121.0,0.0,...,54.0,98.0,0.0,0.0,0.0,112.0,251.0,8146.0,0.0,0.0
2023-03-07,0.0,0.0,6001.0,0.0,0.0,0.0,115.0,0.0,1802.0,295.0,...,488.0,679.0,0.0,0.0,1472.0,104.0,170.0,7063.0,1938.0,0.0
2023-03-08,16.0,15.0,6442.0,0.0,0.0,93.0,93.0,0.0,1169.0,10.0,...,374.0,542.0,0.0,0.0,0.0,9859.0,140.0,5808.0,0.0,62.0


In [ ]:
px.line(d1_df)

I check that data is indeed cumulative as one would expect from Covid confirmed cases. Yet, there's some negative changes from one date to the other, so I plot all these changes on the time span to see whether, for each country a pattern can be found. I also associate a code ranging from 0(Sunday)to 6(Saturday) to the date of the negative change to spot any association with weekday.


In [ ]:
Neg_Changes=(d1_df<0).any(axis=0).loc[(d1_df<0).any(axis=0)]#Countries for which there's has been at least 1 negative change
Neg_Changes=d1_df.loc[:,d1_df.columns.isin(Neg_Changes.index)]
fig=go.Figure()
for i in Neg_Changes.columns:
  fig.add_trace(go.Scatter(x=Neg_Changes[i][Neg_Changes[i]<0].index, y=Neg_Changes[i][Neg_Changes[i]<0].values,name=i
                           ,text=[d.weekday() for d in Neg_Changes[i][Neg_Changes[i]<0].index],mode='markers',visible="legendonly"))
fig.show()

In [ ]:
df_daily=df.copy()


The data at country-level do not show any immediate pattern and, hence, I think it is most likely that these negative changes are the result of corrections that remove a bunch of false positive tests once in a while. Therefore, I believe the data is cumulative.


To kill 2 birds with a stone I aggregate data at weekly frequency, in a way that these corrections are incorporated in the larger timeframe and we should see little to no negative difference in number of confirmed Covid cases.

In [ ]:
df=df.resample('W').max() #Resample Weekly
d1_df=df.diff().iloc[1:,:] #Difference from one week to the other
Neg_Changes=(d1_df<0).any(axis=0).loc[(d1_df<0).any(axis=0)]#Countries for which there's has been at least 1 negative change
Neg_Changes=d1_df.loc[:,d1_df.columns.isin(Neg_Changes.index)]
fig=go.Figure()
for i in Neg_Changes.columns:
  fig.add_trace(go.Scatter(x=Neg_Changes[i][Neg_Changes[i]<0].index, y=Neg_Changes[i][Neg_Changes[i]<0].values,name=i
                           ,text=[d.weekday() for d in Neg_Changes[i][Neg_Changes[i]<0].index],mode='markers',visible="legendonly"))
fig.show() #Plot negative changes now that most corrections have been incorporated by decreasing data granularity from daily to weekly

In [ ]:
#px.line(df)
#px.line(d1_df)

In [ ]:
Z={}
for i in Neg_Changes.columns:
  Z[i]=[*Neg_Changes[i][Neg_Changes[i]<0].index]
for i in Z:
  target=df.iloc[int(np.where(df.index==Z[i][0])[0])-1,:][i]
  temp=df.iloc[int(np.where(df.index==Z[i][0])[0])+1:,:][i]
  df.loc[Z[i][0]:temp[temp>target].index[0],i]=temp[temp>target].min()

In [ ]:
def non_decreasing(L):
    return all(x<=y for x, y in zip(L, L[1:]))
pd.Series([non_decreasing(df[i]) for i in df.columns]).unique()

array([ True])

In [ ]:
px.line(df)

In [ ]:
px.line(df.diff())

In [ ]:
df.to_csv('Confirmed_Cases_Dataset.csv',index=True)

#@title Now we look at the doubling time (Question2)



In [ ]:
df_log, df_log_1d = np.log(1+df),np.log(1+df).diff().iloc[1:,:]

In [ ]:
px.line(df_log)

In [ ]:
df_log_1d

Country/Region,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Czechia,Estonia,...,Slovakia,Slovenia,Spain,Sweden,Switzerland,Ukraine,Denmark,France,Netherlands,United Kingdom
2020-02-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.693147,0.693147,0.000000,0.000000,0.000000,0.559616,0.000000,1.098612
2020-02-09,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.405465,0.000000,0.000000,0.000000,0.000000,0.538997,0.000000,1.609438
2020-02-16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080043,0.000000,0.236389
2020-02-23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.422857
2020-03-01,0.000000,0.000000,2.079442,0.693147,0.405465,0.000000,0.000000,2.079442,1.386294,0.693147,...,0.000000,0.000000,3.344039,2.014903,3.332205,0.000000,1.609438,2.310248,2.944439,1.186581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-12,0.000132,0.000209,0.004854,0.000000,0.001180,0.000149,0.000211,0.000260,0.000833,0.000508,...,0.000377,0.000914,0.000610,0.000473,0.000208,0.000766,0.000278,0.000574,0.000315,0.000911
2023-02-19,0.000176,0.000125,0.005846,0.000000,0.001557,0.000137,0.000255,0.000292,0.001249,0.000542,...,0.000483,0.001323,0.000512,0.000367,0.000226,0.001057,0.000262,0.000577,0.000390,0.001049
2023-02-26,0.000191,0.000000,0.006537,0.000000,0.001991,0.000120,0.000361,0.000056,0.001510,0.000495,...,0.000496,0.001516,0.000536,0.000248,0.000279,0.000114,0.000252,0.000691,0.000515,0.001164
2023-03-05,0.000147,0.000188,0.006586,0.000000,0.002147,0.000152,0.000304,0.000257,0.001301,0.000449,...,0.000543,0.001777,0.000515,0.000262,0.000313,0.001420,0.000215,0.000653,0.000469,0.001078


In [ ]:
for c in df_daily.columns:
  for r in range(1,df_daily.shape[0]):
    if df_daily.iloc[r-1][c]>df_daily.iloc[r][c]:
      df_daily.iloc[r][c]=df_daily.iloc[r-1][c]

pd.Series([non_decreasing(df[i]) for i in df.columns]).unique()

array([ True])

In [ ]:
df_daily

Country/Region,Albania,Andorra,Austria,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Croatia,Czechia,Estonia,...,Slovakia,Slovenia,Spain,Sweden,Switzerland,Ukraine,Denmark,France,Netherlands,United Kingdom
2020-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
2020-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
2020-01-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-05,334427,47875,5940935,994037,4727795,401636,1297062,1268992,4614022,615128,...,2666373,1329877,13770429,2698535,4412439,5701743,3403707,38591184,8598043,24556291
2023-03-06,334427,47875,5943417,994037,4727795,401636,1297243,1269326,4614143,615128,...,2666427,1329975,13770429,2698535,4412439,5701855,3403958,38599330,8598043,24556291
2023-03-07,334427,47875,5949418,994037,4727795,401636,1297358,1269326,4615945,615423,...,2666915,1330654,13770429,2698535,4413911,5701959,3404128,38606393,8599981,24556291
2023-03-08,334443,47890,5955860,994037,4727795,401729,1297451,1269326,4617114,615433,...,2667289,1331196,13770429,2698535,4413911,5711818,3404268,38612201,8599981,24556353


In [ ]:
df_daily.rename(columns={'Country/Region':'Date'}).to_csv('Confirmed_Cases_Daily.csv')